<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Structure-cheat-sheet" data-toc-modified-id="Structure-cheat-sheet-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Structure cheat sheet</a></span></li><li><span><a href="#Data-structure" data-toc-modified-id="Data-structure-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data structure</a></span></li><li><span><a href="#get-features" data-toc-modified-id="get-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>get features</a></span></li><li><span><a href="#Exploration-of-non-spectral-Features" data-toc-modified-id="Exploration-of-non-spectral-Features-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploration of non-spectral Features</a></span></li><li><span><a href="#Exploring-spectral-features" data-toc-modified-id="Exploring-spectral-features-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Exploring spectral features</a></span></li></ul></div>

# Basics

## Structure cheat sheet

1. func: train data lead (following order)
    1. read the descriptive dataframe from the feature-pipeline
    2. extract feature from the feature-objects which are labeled train-dataset from dataframe
    3. create numpy feature array for the processing pipeline
2. preprocessing
    1. Transformation (any combination of the following)
        + log-transform
        + PCA
        + others
    2. Scaling (one of the following)
        + StandardScaler
        + MinMaxScaler
3. Unsupervised Clustering
    1. Estimate initial hyperparameter
    2. Create grid over various hyperparameters
    3. Train all and choose the best according to metric
    
    
in all steps the cluster-recorder object (possibly dataframe-row) will record all the meta-information like hyper-parameters

## Data structure

There are multiple degrees of freedom in the data:

1. Signal to noise ratio (SNR)
2. Machine type
    1. pump
    2. fan
    3. valve (solenoid)
    4. slider
3. Machine ID
    1. four different machine IDs
    
The pipeline will be applied to fixed SNR, fixed machine type and fixed ID

## get features

Get the descriptive dataframe for the features.

The descriptive dataframe contains all IDs of the pump. We will focus on ID '00' for now since the modeling phase is seperated per SNR, per machine, per ID anyway.

class: 
+ uni\_\<model\>
attributes:
+ default threshold
+ roc_auc
methods:
+ fit
+ predict
+ predict_score
+ eval_roc_auc

In [30]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from tqdm.auto import tqdm
import glob

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load tensorflow models
Load detection_pipe


The exploration of the Dimensionality Reduction was already done in another notebook. We derived the following rules of thumb:

1. PCA and ICA deliver almost the same results looking at the relative absolute error
2. PCA is usually much faster
3. on PSD and the ICA demix matrix, no dimensionality reduction is needed
4. for a framed Mel-spectrum a number of components between 32 and 64 is a good measure. The resulting error is about 2-4%
5. for a whole Mel-spectrum a number of components between 64 and 128 is advised

## Exploration of non-spectral Features

## Exploring spectral features

In [31]:
diagram = 'extdia_v1'
machines = ['pump', 'fan', 'slider', 'valve']
SNRs = ['6dB', 'min6dB']
IDs = ['00', '02', '04', '06']

features = [('MEL_den', {'function':'frame', 'frames':5})]

tasks = [{'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1_essential/{}{}{}_EDiaV1HP'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_AutoEncoder, {'epochs':50})

pipes = [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

features = [('MEL_den', {'function':'frame', 'frames':5})]

tasks += [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1_essential/{}{}{}_EDiaV1HP'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {'n_estimators':64, 'max_features':4})

pipes += [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

features = [('PSD_raw', {'function':'channel'})]

tasks += [{
        'path_descr':glob.glob(BASE_FOLDER 
                            + '/dataset/extdia_v1_essential/{}{}{}_EDiaV1HP'.format(machine, SNR, ID) 
                            + "*pandaDisc*.pkl", recursive=True)[0],
        'feat':feature[1], 
        'feat_col':feature[0], 
        'SNR':SNR, 
        'machine':machine, 
        'ID':ID,
        'BASE_FOLDER':BASE_FOLDER}
        for machine in machines
        for SNR in SNRs
        for ID in IDs
        for feature in features
        ]

preprocessing = [
    (PCA, {'n_components':64}),
    (StandardScaler, {})
]

modeling = (uni_IsolationForest, {'n_estimators':200, 'max_features':1})

pipes += [Pipe(preprocessing, modeling) for i in range(len(features)*len(IDs)*len(machines)*len(SNRs))]

In [34]:
#failed = 3, 7
start = 0
for pipe, task in tqdm(zip(pipes[start:], tasks[start:]), total=len(tasks[start:])):
    pipe.run_pipe(task)

../..//dataset/extdia_v1\valve6dB00_EDiaV1HPTsSl_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
Epoch 1/50
527/527 [==============================] - 5s 10ms/step - loss: 0.9304
Epoch 2/50
527/527 [==============================] - 4s 7ms/step - loss: 0.8376
Epoch 3/50
527/527 [==============================] - 4s 7ms/step - loss: 0.7817
Epoch 4/50
527/527 [==============================] - 4s 7ms/step - loss: 0.7467
Epoch 5/50
527/527 [==============================] - 4s 7ms/step - loss: 0.7225
Epoch 6/50
527/527 [==============================] - 4s 7ms/step - loss: 0.7052
Epoch 7/50
527/527 [==============================] - 3s 7ms/step - loss: 0.6920
Epoch 8/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6817
Epoch 9/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6735
Epoch 10/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6665
Epoc

355/355 [==============================] - 2s 6ms/step - loss: 0.6009
Epoch 46/50
355/355 [==============================] - 2s 6ms/step - loss: 0.5994
Epoch 47/50
355/355 [==============================] - 2s 6ms/step - loss: 0.5985
Epoch 48/50
355/355 [==============================] - 2s 6ms/step - loss: 0.5977
Epoch 49/50
355/355 [==============================] - 2s 6ms/step - loss: 0.5968
Epoch 50/50
355/355 [==============================] - 2s 6ms/step - loss: 0.5960
model fitted successfully

...evaluating model
evaluation successfull, roc_auc: 0.9100323915520133
pipe saved to pickle
../..//dataset/extdia_v1\valve6dB04_EDiaV1HPTsSl_pandaDisc.pkl --> Done
...loading data
data loading completed

...preprocessing data
data preprocessing finished

...fitting the model
Epoch 1/50
532/532 [==============================] - 5s 10ms/step - loss: 0.9410
Epoch 2/50
532/532 [==============================] - 3s 6ms/step - loss: 0.8461
Epoch 3/50
532/532 [==============================] -

527/527 [==============================] - 3s 6ms/step - loss: 0.6072
Epoch 38/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6060
Epoch 39/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6047
Epoch 40/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6033
Epoch 41/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6021
Epoch 42/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6012
Epoch 43/50
527/527 [==============================] - 3s 6ms/step - loss: 0.6002
Epoch 44/50
527/527 [==============================] - 3s 6ms/step - loss: 0.5990
Epoch 45/50
527/527 [==============================] - 3s 6ms/step - loss: 0.5981
Epoch 46/50
527/527 [==============================] - 3s 7ms/step - loss: 0.5971
Epoch 47/50
527/527 [==============================] - 4s 7ms/step - loss: 0.5964
Epoch 48/50
527/527 [==============================] - 3s 7ms/step - loss: 0.5954
Epoch 49/50
527/527 [=======

355/355 [==============================] - 2s 7ms/step - loss: 0.6722
Epoch 30/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6704
Epoch 31/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6686
Epoch 32/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6674
Epoch 33/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6662
Epoch 34/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6649
Epoch 35/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6641
Epoch 36/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6626
Epoch 37/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6611
Epoch 38/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6600
Epoch 39/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6593
Epoch 40/50
355/355 [==============================] - 2s 7ms/step - loss: 0.6581
Epoch 41/50
355/355 [=======

527/527 [==============================] - 4s 7ms/step - loss: 0.6877
Epoch 22/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6861
Epoch 23/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6838
Epoch 24/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6821
Epoch 25/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6797
Epoch 26/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6779
Epoch 27/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6765
Epoch 28/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6747
Epoch 29/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6734
Epoch 30/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6724
Epoch 31/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6711
Epoch 32/50
527/527 [==============================] - 4s 7ms/step - loss: 0.6699
Epoch 33/50
527/527 [=======